In [1]:
from tensorflow.keras.models import load_model
from tensorflow.keras import mixed_precision 
mixed_precision.set_global_policy('mixed_float16')
import geopandas as gps
import rasterio                  # I/O raster data (netcdf, height, geotiff, ...)
import rasterio.warp             # Reproject raster samples
from rasterio import windows
from rasterio import features
import fiona                     # I/O vector data (shape, geojson, ...)
import pyproj                    # Change coordinate reference system
from osgeo import gdal, ogr, osr
import pandas as pd
import shapely
from shapely.geometry import Point, Polygon
from shapely.geometry import mapping, shape
import cv2
import json
import geopandas as gpd
import numpy as np               # numerical array manipulation
import pandas as pd
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from tqdm import tqdm
from PIL import Image
import PIL.ImageDraw

from itertools import product

import sys
from core.UNet import UNet
from core.losses import tversky, focalTversky, bce_dice_loss, accuracy, dice_loss, IoU, recall, precision
from core.optimizers import adaDelta, adagrad, adam, nadam
from core.visualize import display_images
# from core.dataset_generator import DataGenerator

import matplotlib.pyplot as plt  # plotting tools
%matplotlib inline
import warnings                  # ignore annoying warnings
warnings.filterwarnings("ignore")
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

%reload_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

os.environ['TF_ENABLE_AUTO_MIXED_PRECISION'] = '1'
import tensorflow as tf
print(tf.__version__)

2023-11-18 15:11:53.253398: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA RTX A6000, compute capability 8.6


2023-11-18 15:11:54.188180: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-11-18 15:11:54.188786: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-11-18 15:11:54.200664: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:17:00.0 name: NVIDIA RTX A6000 computeCapability: 8.6
coreClock: 1.8GHz coreCount: 84 deviceMemorySize: 47.53GiB deviceMemoryBandwidth: 715.34GiB/s
2023-11-18 15:11:54.200685: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2023-11-18 15:11:54.202412: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2023-11-18 15:11:54.202460: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2023-1

2.4.1


In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto(
    #device_count={"CPU": 64},
    allow_soft_placement=True, 
    log_device_placement=False)
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

2023-11-18 15:11:54.877693: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-18 15:11:54.879562: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-11-18 15:11:54.879837: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:17:00.0 name: NVIDIA RTX A6000 computeCapability: 8.6
coreClock: 1.8GHz coreCount: 84 deviceMemorySize: 47.53GiB deviceMemoryBandwidth: 715.34GiB/s
2023-11-18 15:11:54.879869: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2023-11-18 15:11:54.879900: I tensorflow/stream_executor/platform/default/dso_loader.cc

In [10]:
# Required configurations (including the input and output paths) are stored in a separate file (such as config/RasterAnalysis.py)
# Please provide required info in the file before continuing with this notebook. 
 
from config import RasterAnalysis_png
# In case you are using a different folder name such as configLargeCluster, then you should import from the respective folder 
# Eg. from configLargeCluster import RasterAnalysis
config = RasterAnalysis_png.Configuration()

self.trained_model_path: /media/nkd/backup/5_lakemapping/U_Net/saved_models/UNet/lakes_20231117-1542_AdaDelta_dice_loss_012345_512.h5


In [4]:
# Load a pretrained model 
OPTIMIZER = adaDelta
# OPTIMIZER = adam
OPTIMIZER=mixed_precision.LossScaleOptimizer(OPTIMIZER)
model = load_model(config.trained_model_path, custom_objects={'dice loss': dice_loss, 'accuracy':accuracy ,'recall':recall, 'precision':precision,'IoU': IoU}, compile=False)
model.compile(optimizer=OPTIMIZER, loss=dice_loss, metrics=[dice_loss, accuracy,recall, precision, IoU])

2023-11-18 15:11:55.324118: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-11-18 15:11:55.324381: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:17:00.0 name: NVIDIA RTX A6000 computeCapability: 8.6
coreClock: 1.8GHz coreCount: 84 deviceMemorySize: 47.53GiB deviceMemoryBandwidth: 715.34GiB/s
2023-11-18 15:11:55.324415: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2023-11-18 15:11:55.324447: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2023-11-18 15:11:55.324453: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2023-11-18 15:11:55.324460: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcufft.so.10
2023

In [5]:
# Methods to add results of a patch   to    the total results of a larger area. 
#The operator could be min (useful if there are too many false positives), max (useful for tackle false negatives)
#res:mask [rows,cols] predition=np.squeeze(prediction[i], axis = -1) (col, row, wi, he) = batch_pos[i]
def addTOResult(res, prediction, row, col, he, wi,operator,ie_width):
    
    currValue = res[row+ie_width:row+he-ie_width, col+ie_width:col+wi-ie_width]
    newPredictions = prediction[ie_width:he-ie_width, ie_width:wi-ie_width]
    
# IMPORTANT: MIN can't be used as long as the mask is initialed with 0!!!!! 
#If you want to use MIN initial the mask with -1 and handle the case of default value(-1) separately.
    if operator == 'min': # Takes the min of current prediction and new prediction for each pixel  
        currValue [currValue == -1] = 1 #Replace -1 with 1 in case of MIN  
        res[row+ie_width:row+he-ie_width, col+ie_width:col+wi-ie_width] = np.minimum(currValue, newPredictions)
    elif operator == 'max':
        res[row+ie_width:row+he-ie_width, col+ie_width:col+wi-ie_width] = np.maximum(currValue, newPredictions)
    else:
        res[row+ie_width:row+he-ie_width, col+ie_width:col+wi-ie_width] = newPredictions  
    return (res)

In [6]:
def readBands(fn,window):
    img=rasterio.open(fn)
    im=img.read(window=window)/1000
    return im

# Methods that actually makes the predictions
def predict_using_model(model, batch, batch_pos, mask,operator,ie_width):
#     print(batch)
    tm = np.stack(batch, axis = 0)
#     print(tm)
    prediction = model.predict(tm)
    for i in range(len(batch_pos)): 
        (col, row, wi, he) = batch_pos[i]
        p = np.squeeze(prediction[i], axis = -1)
        # Instead of replacing the current values with new values, use the user specified operator (MIN,MAX,REPLACE)
        mask = addTOResult(mask, p, row, col, he, wi,operator,ie_width)  
    return mask
    
def detect_lake(fullPath,NDWI_img,operator, width=512, height=512, stride = 256,ie_width=0,bandNum=1):
    nols, nrows = NDWI_img.meta['width'], NDWI_img.meta['height']
    meta = NDWI_img.meta.copy() 
    if 'float' not in meta['dtype']: #The prediction is a float so we keep it as float to be consistent with the prediction. 
        meta['dtype'] = np.float32
    col_index=list(range(0, nols-width, stride))
    col_index.append(nols-width)
    row_index=list(range(0, nrows-height, stride))
    row_index.append(nrows-height)
    offsets = product(col_index,row_index)
    big_window = windows.Window(col_off=0, row_off=0, width=nols, height=nrows)
    print('the size of current NDWI_img',nrows, nols) 

    mask = np.zeros((nrows, nols), dtype=meta['dtype'])

#     mask = mask -1   # Note: The initial mask is initialized with -1 instead of zero   to handle the MIN case (see addToResult)
    batch = []
    batch_pos = [ ]
    for col_off, row_off in  tqdm(offsets):
        window =windows.Window(col_off=col_off, row_off=row_off, width=width, height=height).intersection(big_window)
        transform = windows.transform(window, NDWI_img.transform) 
#         hbh:notice datatype is float
        patch = np.full((height, width, bandNum),-1.0)
        if bandNum==1:
            read_NDWI_img=NDWI_img.read(window=window)
            temp_im=read_NDWI_img
        elif bandNum==3:
            read_NDWI_img=NDWI_img.read(window=window)/100
            read_green_img =readBands(fullPath.replace(config.ndwi_fn_st ,config.green_fn_st),window)
            read_swir_img = readBands(fullPath.replace(config.ndwi_fn_st ,config.swir_fn_st),window)
            temp_im =  np.concatenate((read_NDWI_img,read_green_img,read_swir_img), axis=0)
        else:#bandNum==5
            read_NDWI_img=NDWI_img.read(window=window)/100
            read_red_img =readBands(fullPath.replace(config.ndwi_fn_st ,config.red_fn_st),window)
            read_green_img =readBands(fullPath.replace(config.ndwi_fn_st ,config.green_fn_st),window)
            read_blue_img = readBands(fullPath.replace(config.ndwi_fn_st ,config.blue_fn_st),window)
            read_swir_img = readBands(fullPath.replace(config.ndwi_fn_st ,config.swir_fn_st),window)
            temp_im =  np.concatenate((read_NDWI_img,read_red_img,read_green_img,read_blue_img, read_swir_img), axis=0)

        temp_im =  np.transpose(temp_im, axes=(1,2,0))
    
        patch[:window.height, :window.width] = temp_im   
        
        batch.append(patch)
        batch_pos.append((window.col_off, window.row_off, window.width, window.height))
        if (len(batch) == config.BATCH_SIZE):
            mask = predict_using_model(model, batch, batch_pos, mask,operator,ie_width)
            batch = []
            batch_pos = []
            
    # To handle the edge of images as the image size may not be divisible by n complete batches and few frames on the edge may be left.
    if batch:
        mask = predict_using_model(model, batch, batch_pos, mask,operator,ie_width)
        batch = []
        batch_pos = []

    return(mask, meta)

In [7]:
def raster2vector(raster_path, vecter_path, field_name="class", ignore_values = None):
    
    # 读取路径中的栅格数据
    raster = gdal.Open(raster_path)
    # in_band 为想要转为矢量的波段,一般需要进行转矢量的栅格都是单波段分类结果
    # 若栅格为多波段,需要提前转换为单波段
    band = raster.GetRasterBand(1)
    
    # 读取栅格的投影信息,为后面生成的矢量赋予相同的投影信息
    prj = osr.SpatialReference()
    prj.ImportFromWkt(raster.GetProjection())
    
    
    drv = ogr.GetDriverByName("ESRI Shapefile")
    # 若文件已经存在,删除
    if os.path.exists(vecter_path):
        drv.DeleteDataSource(vecter_path)
        
    # 创建目标文件
    polygon = drv.CreateDataSource(vecter_path)
    # 创建面图层
    poly_layer = polygon.CreateLayer(vecter_path[:-4], srs=prj, geom_type=ogr.wkbMultiPolygon)
    # 添加浮点型字段,用来存储栅格的像素值
    field = ogr.FieldDefn(field_name, ogr.OFTReal)  
    poly_layer.CreateField(field)
    
    # FPolygonize将每个像元转成一个矩形，然后将相似的像元进行合并
    # 设置矢量图层中保存像元值的字段序号为0
    gdal.FPolygonize(band, None, poly_layer, 0)
    
    # 删除ignore_value链表中的类别要素
    if ignore_values is not None:
        for feature in poly_layer:
            class_value = feature.GetField('class')
            for ignore_value in ignore_values:
                if class_value==ignore_value:
                    # 通过FID删除要素
                    poly_layer.DeleteFeature(feature.GetFID())
                    break
                
    polygon.SyncToDisk()
    polygon = None
    print('Vector File Exported Successfully!')
        
def writeMaskToDisk(detected_mask, detected_meta, wp, write_as_type = 'uint8', th = 0.5, create_countors = False):
    # Convert to correct required before writing
    if 'float' in str(detected_meta['dtype']) and 'int' in write_as_type:
        print(f'Converting prediction from {detected_meta["dtype"]} to {write_as_type}, using threshold of {th}')#float32 to uint8
#         initial code have problem of big lake
        detected_mask[detected_mask<th]=0
        detected_mask[detected_mask>=th]=1
        detected_mask = detected_mask.astype(write_as_type)#'uint8'
        detected_meta['dtype'] =  write_as_type
    
    # compress tif
    detected_meta.update({"compress": 'lzw'})
    
    with rasterio.open(wp, 'w', **detected_meta) as outds:
        outds.write(detected_mask, 1)

In [ ]:
# Predict trees in the all the files in the input image dir 
# Depending upon the available RAM, images may not to be split before running this cell.
# Use the Auxiliary-2-SplitRasterToAnalyse if the images are too big to be analysed in memory.
all_files = []

# for root, dirs, files in os.walk(config.input_image_dir):
#     for file in files:
#         if file.endswith(config.input_image_type)and file.startswith(config.ndwi_fn_st):
#              all_files.append((os.path.join(root, file), file))

for i in range(0,config.type_num):
    for root, dirs, files in os.walk(os.path.join(config.input_image_dir,'test/type{}'.format(i))):
        for file in files:
            if file.endswith(config.input_image_type)and file.startswith(config.ndwi_fn_st):
                 all_files.append((os.path.join(root, file), file))
print(all_files)
                                     
operator='max'
suffix='_iew{}'.format(config.ignore_edge_width)
for fullPath, filename in all_files:
    output = filename.replace(config.input_image_type,config.output_image_type)
    outputFile = os.path.join(config.output_dir, 'test/'+output.replace(config.ndwi_fn_st, config.output_prefix + operator + suffix))
    if not os.path.isfile(outputFile) or config.overwrite_analysed_files:
        with rasterio.open(fullPath) as NDWI:
            print(fullPath)
            detectedMask, detectedMeta = detect_lake(fullPath,NDWI,operator,width = config.WIDTH, height = config.HEIGHT, stride = config.STRIDE,ie_width=config.ignore_edge_width,bandNum=config.band_num)
            writeMaskToDisk(detectedMask, detectedMeta, outputFile, write_as_type = config.output_dtype, th = 0.5, create_countors = False) 
            vecter_path =outputFile.replace('tif','shp')
            field_name = "class"
#            第0类删除,若实际情况不需要1类和2类,则ignore_values = [1,2]
            ignore_values = [0]
            raster2vector(outputFile, vecter_path, field_name=field_name, ignore_values = ignore_values)#, ignore_values = ignore_values
    else:
        print('File already analysed!', fullPath)
        
print('finish')

[('/media/nkd/backup/5_lakemapping/sample600/patchesReshape/test/type0/ndwi_11490.png', 'ndwi_11490.png'), ('/media/nkd/backup/5_lakemapping/sample600/patchesReshape/test/type0/ndwi_3922.png', 'ndwi_3922.png'), ('/media/nkd/backup/5_lakemapping/sample600/patchesReshape/test/type0/ndwi_6723.png', 'ndwi_6723.png'), ('/media/nkd/backup/5_lakemapping/sample600/patchesReshape/test/type0/ndwi_6485.png', 'ndwi_6485.png'), ('/media/nkd/backup/5_lakemapping/sample600/patchesReshape/test/type0/ndwi_343.png', 'ndwi_343.png'), ('/media/nkd/backup/5_lakemapping/sample600/patchesReshape/test/type0/ndwi_9080.png', 'ndwi_9080.png'), ('/media/nkd/backup/5_lakemapping/sample600/patchesReshape/test/type0/ndwi_1727.png', 'ndwi_1727.png'), ('/media/nkd/backup/5_lakemapping/sample600/patchesReshape/test/type0/ndwi_5085.png', 'ndwi_5085.png'), ('/media/nkd/backup/5_lakemapping/sample600/patchesReshape/test/type0/ndwi_178.png', 'ndwi_178.png'), ('/media/nkd/backup/5_lakemapping/sample600/patchesReshape/test/t

54it [00:02, 20.28it/s]


Converting prediction from float32 to uint8, using threshold of 0.5
Vector File Exported Successfully!
/media/nkd/backup/5_lakemapping/sample600/patchesReshape/test/type0/ndwi_3922.png
the size of current NDWI_img 2015 2032


49it [00:02, 19.28it/s]


Converting prediction from float32 to uint8, using threshold of 0.5
Vector File Exported Successfully!
/media/nkd/backup/5_lakemapping/sample600/patchesReshape/test/type0/ndwi_6723.png
the size of current NDWI_img 2217 4747


...

128it [00:06, 18.68it/s]


Converting prediction from float32 to uint8, using threshold of 0.5
Vector File Exported Successfully!
/media/nkd/backup/5_lakemapping/sample600/patchesReshape/test/type0/ndwi_3272.png
the size of current NDWI_img 3108 4185


192it [00:10, 18.16it/s]


Converting prediction from float32 to uint8, using threshold of 0.5
Vector File Exported Successfully!
/media/nkd/backup/5_lakemapping/sample600/patchesReshape/test/type0/ndwi_6305.png
the size of current NDWI_img 5794 4961


151it [00:07, 25.11it/s]